In [1]:
import openai
def setup_openai_api_key():
    from dotenv import load_dotenv
    _ = load_dotenv()
setup_openai_api_key()

LLM_MODEL:str = "gpt-4o-mini"

# LangChain

LangChain is an framework designed to simplify LLM application development. It allows to chain LLM calls, prompts, tools, and other components together into sequential workflows.

## Module

> https://python.langchain.com/docs/concepts/architecture/

- `langchain-core`: Base abstractions for chat models and other components.
- `langchain-openai`, `langchain-anthropic`: Important integrations have been split into lightweight packages.
- `langchain`: Chains, agents, and retrieval strategies that make up an application's cognitive architecture.
- `langchain-community`: Third-party integrations that are community maintained.
- `langgraph`: Orchestration framework for combining LangChain components into production-ready applications with persistence, streaming, and other key features.

### Installation

**With [uv](https://docs.astral.sh/uv/getting-started/installation/)**

```
uv add langchain
```

**With pip**

```
pip install langchain
```



## Chat Model

LLMs commonly takes `messages` as input and returns a `message` as output. 

```mermaid
flowchart LR
    a[/messages/] --> b[LLM]
    b --> c[/message/]
```

Modern LLM offers additional capabilities:

|Capability|Description|
|----------|-----------|
|**Tool calling**    |Ability to find a suitable function along with arguments.
|**Structure output**|Ability to return structured format such as json matching a given schema.
|**Muldimodality**   |Ability to work with data other than text such as image, audio and video.

### Standard parameters

|Parameter|Description|
|---------|-----------|
model     |The identifier of AI model to use (i.e. "gpt-4")
temperature|Controls the randomness of the model's output. A higher value (e.g., 1.0) makes responses more creative, while a lower value (e.g., 0.0) makes them more deterministic and focused.
timeout   |The maximum time (in seconds) to wait for a response from the model before canceling the request.
max_tokens|Limits the total number of tokens in the response. This controls how long the output can be.
stop      |Specifies stop sequences that indicate when the model should stop generating tokens. For example, you might use specific strings to signal the end of a response.
max_retries|The maximum number of attempts the system will make to resend a request if it fails due to issues like network timeouts or rate limits.
api_key    |The API key required for authenticating with the model provider.
base_url   |The URL of the API endpoint where requests are sent. This is typically provided by the model's provider and is necessary for directing your requests.
rate_limiter|An optional BaseRateLimiter to space out requests to avoid exceeding rate limits. See rate-limiting below for more details.

## Message

> https://python.langchain.com/docs/concepts/messages/

Messages are the unit of communication in chat models.

**Message represents:**
- input and output of a chat model
- any additional context
- metadata that may be associated with a conversation

A message typically consists of the following information:
- **Role**: The role of the message (i.e. "user", "assistant").
- **Content**: The content of the message (i.e. text, multimodal data).
- **Additional metadata**: id, name, token usage and other model-specific metadata.

**Role**

Roles are used to distinguish between different types of messages in a conversation and help the chat model understand how to respond to a given sequence of messages.

|Role  |Description|
|------|-----------|
system<br/>`SystemMessage`| Tell the LLM how to behave and provide additional context. Not supported by all chat model providers.
user<br/>`HumanMessage`   | Represents input from a user interacting with the model, usually in the form of text or other interactive input.
assistant<br/>`AIMessage` | Represents a response from the model, which can include text or a request to invoke tools.
tool<br/>`ToolMessage`    | A message used to pass the results of a tool invocation back to the model after external data or processing has been retrieved. Used with chat models that support tool calling.

**Content**

The content of a message text or a list of dictionaries representing multimodal data. The exact format of the content can vary between different chat model providers.

**LangChain Messages**

LangChain provides a unified message format that can be used across all chat models, allowing users to work with different chat models without worrying about the specific details of the message format used by each model provider.

The five main message types are:
|Type|Description|
|----|-----------|
|`SystemMessage` | corresponds to system role
|`HumanMessage`  | corresponds to user role
|`AIMessage`     | corresponds to assistant role
|`ToolMessage`   | corresponds to tool role

**OpenAI Message Format**

In [18]:
# OpenAI Message Format 
import openai
from openai.types.chat import (
    ChatCompletionSystemMessageParam, ChatCompletionUserMessageParam, ChatCompletionAssistantMessageParam
)
# response API
respond = openai.OpenAI().responses.create(     
    model=LLM_MODEL,
    instructions="You are a helpful assistant", # system message
    input = "Hello World!",                     # user message
    temperature=0
)
print(respond)

system_msg: ChatCompletionSystemMessageParam = {
    "role": "system",
    "content": "You are a helpful assistant"
}
human_msg: ChatCompletionUserMessageParam = {
    "role": "user",
    "content": "Hello World!"
}
ai_msg: ChatCompletionAssistantMessageParam = {
    "role": "assistant",
    "content": "This is message from LLM!"
}
# chat completion API
respond = openai.OpenAI().chat.completions.create(
    model=LLM_MODEL,
    messages=[system_msg, human_msg],
    temperature=0
)
print(respond)


Response(id='resp_68bfd7fa93708196be44e400856c1bfa02d1f141ae65216b', created_at=1757403130.0, error=None, incomplete_details=None, instructions='You are a helpful assistant', metadata={}, model='gpt-4o-mini-2024-07-18', object='response', output=[ResponseOutputMessage(id='msg_68bfd7fb81d4819683db710cbb2c7c1702d1f141ae65216b', content=[ResponseOutputText(annotations=[], text='Hello! How can I assist you today?', type='output_text', logprobs=[])], role='assistant', status='completed', type='message')], parallel_tool_calls=True, temperature=0.0, tool_choice='auto', tools=[], top_p=1.0, background=False, max_output_tokens=None, max_tool_calls=None, previous_response_id=None, prompt=None, prompt_cache_key=None, reasoning=Reasoning(effort=None, generate_summary=None, summary=None), safety_identifier=None, service_tier='default', status='completed', text=ResponseTextConfig(format=ResponseFormatText(type='text'), verbosity='medium'), top_logprobs=0, truncation='disabled', usage=ResponseUsage(i

**LangChain Message Format**

In [ ]:
from langchain_core.messages import (
    SystemMessage, HumanMessage, AIMessage, ToolMessage
)
from langchain_openai import OpenAI

msg_sys = SystemMessage(content="this is system message")
msg_human = HumanMessage(content="this is human message")
msg_ai = AIMessage(content="this is ai message")
print("msg :", [msg_sys, msg_human, msg_ai])
print("type:", [msg_sys.type, msg_human.type, msg_ai.type])

model = OpenAI(model=LLM_MODEL, temperature=0)
respond = model.invoke([msg_sys, msg_human])
print(respond)

[SystemMessage(content='this is system message', additional_kwargs={}, response_metadata={}), HumanMessage(content='this is human message', additional_kwargs={}, response_metadata={}), AIMessage(content='this is ai message', additional_kwargs={}, response_metadata={})]
['system', 'human', 'ai']


## Prompt

Prompt is a template that structures input to language models. It defines how to format user input, system instructions, and context before sending to the LLM.

Prompts ensure consistent, structured communication between your application and the LLM.

Prompt Templates take as input a dictionary, where each key represents a variable in the prompt template to fill in.

**Key components:**
- `ChatPromptTemplate`: formats messages for chat models
- `SystemMessagePrompt`: sets AI behavior/role
- `HumanMessagePrompt`: user input template
- `Variables`: placeholders filled at runtime (e.g., {user_input}, {context})

### Prompt Templates

**String PromptTemplates**

In [19]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("Tell me a joke about {topic}")
prompt_template.invoke({"topic": "cats"})

StringPromptValue(text='Tell me a joke about cats')

**ChatPromptTemplates**

In [20]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate([
    ("system", "You are a helpful assistant"),
    ("user", "Tell me a joke about {topic}")
])
prompt_template.invoke({"topic": "cats"})

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me a joke about cats', additional_kwargs={}, response_metadata={})])

**MessagesPlaceholder**

In [21]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

prompt_template = ChatPromptTemplate([
    ("system", "You are a helpful assistant"),
    MessagesPlaceholder("msgs")
])
# Simple example with one message
prompt_template.invoke({"msgs": [HumanMessage(content="hi!")]})
# More complex example with conversation history
messages_to_pass = [
    HumanMessage(content="What's the capital of France?"),
    AIMessage(content="The capital of France is Paris."),
    HumanMessage(content="And what about Germany?")
]
formatted_prompt = prompt_template.invoke({"msgs": messages_to_pass})
print(formatted_prompt)

messages=[SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content="What's the capital of France?", additional_kwargs={}, response_metadata={}), AIMessage(content='The capital of France is Paris.', additional_kwargs={}, response_metadata={}), HumanMessage(content='And what about Germany?', additional_kwargs={}, response_metadata={})]


### Prompt Pipeline

**Create prompt from template:**

```mermaid
graph LR
    A[template] --> C[Prompt]
```

**Use prompt to get response:**

```mermaid
graph LR
    A[style] --> C[Prompt]
    B[message] --> C

    C --> D[LLM]
```

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage
customer_email = """\
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!\
"""

style = """\
British English \
in a calm and respenctful tone\
"""

prompt = """\
Translate the text \
that is delimited by triple backticks \
into a style that is {style}.
text: ```{customer_email}```
"""

# With PromptTemplate
prompt_template = PromptTemplate.from_template(prompt)
prompt_template.invoke({
    "style": style,
    "customer_email": customer_email
})

# With ChatPromptTemplate
prompt_template = ChatPromptTemplate([
    SystemMessage(content="Translate the user input into a style that is {style}."),
    HumanMessage(content="{customer_email}")
])
prompt_template.invoke({
    "style": style,
    "customer_email": customer_email
})

ChatPromptValue(messages=[SystemMessage(content='Translate the user input into a style that is British English in a calm and respenctful tone.', additional_kwargs={}, response_metadata={}), HumanMessage(content="Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!", additional_kwargs={}, response_metadata={})])

## Runnable

The Runnable interface is the fundamental component of LangChain, and it's implemented across many of them, such as language models, output parsers, retrievers, compiled LangGraph graphs and more.

### Core interface

- `invoke`: A single input is transformed into an output.
- `batch`: Multiple inputs are efficiently transformed into outputs.
- `stream`: Outputs are streamed as they are produced.
- `inspect`: Schematic information about Runnable's input, output, and configuration can be accessed.
- `compose`: Multiple Runnables can be composed to work together using the LangChain Expression Language (LCEL) to create complex pipelines.
 
### Input and output types

Every Runnable is characterized by an input and output type. These input and output types are defined by the Runnable itself.

Runnable methods that result in the execution of the Runnable (e.g., invoke, batch, stream, astream_events) work with these input and output types.

- `invoke`: Accepts an input and returns an output.
- `batch`: Accepts a list of inputs and returns a list of outputs.
- `stream`: Accepts an input and returns a generator that yields outputs.

**Input and output of each component**

|Component|Input Type|Output Type|
|---------|----------|-----------|
|Prompt   |dictionary|PromptValue|
|ChatModel|a string, list of chat messages or a PromptValue|ChatMessage|
|LLM      |a string, list of chat messages or a PromptValue|String|
|OutputParser|the output of an LLM or ChatModel|Depends on the parser|
|Retriever|a string|List of Documents|
|Tool     |a string or dictionary, depending on the tool|Depends on the tool|

### Inspect Schema

Runnable provides methods to get json or Pydantic schema of input, output and config.

|Method|Description|
|------|-----------|
|`get_input_schema`|Gives the Pydantic Schema of the input schema for the Runnable|
|`get_output_schema`|Gives the Pydantic Schema of the output schema for the Runnable|
|`config_schema`|Gives the Pydantic Schema of the config schema for the Runnable|
|`get_input_jsonschema`|Gives the JSONSchema of the input schema for the Runnable|
|`get_output_jsonschema`|Gives the JSONSchema of the output schema for the Runnable|
|`get_config_jsonschema`|Gives the JSONSchema of the config schema for the Runnable|

### RunnableConfig

Any of the runnnable execute methods (i.e. `invoke`) accept a second argument called `RunnableConfig`. This argument is a `dict` that contains configuration that will be used at run time during the execution.

It is important that the `RunnableConfig` is propagated to all sub-calls made by the `Runnable`. This allows providing run time configuration values to the parent `Runnable` that are inherited by all sub-calls.

|Attribute|Description|
|---------|-----------|
|run_name|Name used for the given Runnable (not inherited)|
|run_id  |Unique identifier for this call. sub-calls will get their own unique run ids|
|tags    |Tags for this call and any sub-calls|
|metadata|Metadata for this call and any sub-calls|
|callbacks|Callbacks for this call and any sub-calls|
|max_concurrency|Maximum number of parallel calls to make (e.g., used by batch)|
|recursion_limit|Maximum number of times a call can recurse (e.g., used by Runnables that return Runnables)|
|configurable|Runtime values for configurable attributes of the Runnable|

**Create a `Runnable`:**

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_openai import OpenAI

# With LCEL
prompt_template = PromptTemplate.from_template("My prompt")
model = OpenAI(model=LLM_MODEL, temperature=0)
output_parser = StrOutputParser()
runnable_chain = prompt_template | model | output_parser

# With custom runnable
def foo(input):
    return f"{input} It's me!"
foo_runnable = RunnableLambda(foo)

hello It's me!


**Running a runnable**

In [4]:
from langchain_core.runnables import RunnableLambda

def foo(input):
    return f"{input["key1"]}, It's me! {input["key2"]}"
foo_runnable = RunnableLambda(foo)

result = foo_runnable.invoke({"key1": "val1", "key2": "val2"})
print(result)

val1, It's me! val2


**Setting a runnable config**

In [ ]:
from langchain_core.runnables import RunnableLambda, RunnableConfig

config: RunnableConfig = {
    "run_name": "hello_run",
    "tags": ["tag1", "tag2"],
    "metadata": {"hello_key": "hello_val"}
}

def foo(input):
    return f"{input} It's me!"
foo_runnable = RunnableLambda(foo)

result = foo_runnable.invoke(
    "hello",
    config = config
)

### LCEL Cheatsheet

> https://python.langchain.com/docs/how_to/lcel_cheatsheet/

**Runnable**

In [ ]:
# invoke
from langchain_core.runnables import RunnableLambda
runnable = RunnableLambda(lambda x: str(x))
print("invoke:", runnable.invoke(5))

# batch
from langchain_core.runnables import RunnableLambda
runnable = RunnableLambda(lambda x: str(x))
print("batch:", runnable.batch([7, 8, 9]))

# stream
from langchain_core.runnables import RunnableLambda
def func1(x):
    for y in x:
        yield str(y)
runnable = RunnableLambda(func1)
print("stream:")
for chunk in runnable.stream(range(5)):
    print(chunk)

# Compose runnable with pipe operator
from langchain_core.runnables import RunnableLambda
runnable1 = RunnableLambda(lambda x: {"foo": x})
runnable2 = RunnableLambda(lambda x: [x] * 2)
chain = runnable1 | runnable2
print("compose with pipe:", chain.invoke(2))

# Runnable parallel
from langchain_core.runnables import RunnableLambda, RunnableParallel
runnable1 = RunnableLambda(lambda x: {"foo": x})
runnable2 = RunnableLambda(lambda x: [x] * 2)
chain = RunnableParallel(first=runnable1, second=runnable2)
print("RunnableParallel:", chain.invoke(2))

# Runnable lambda
from langchain_core.runnables import RunnableLambda
def func2(x):
    return x + 5
runnable = RunnableLambda(func2)
print("RunnableLambda:", runnable.invoke(2))

# Merge input and output
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
runnable1 = RunnableLambda(lambda x: x["foo"] + 7)
chain = RunnablePassthrough.assign(bar=runnable1)
print("Merge input and output:", chain.invoke({"foo": 10}))

# Runnable bind
from langchain_core.runnables import RunnableLambda
def func3(main_arg: dict, other_arg: str|None = None) -> dict:
    if other_arg:
        return {**main_arg, **{"foo": other_arg}}
    return main_arg
runnable1 = RunnableLambda(func3)
bound_runnable1 = runnable1.bind(other_arg="bye")
print("Bind:", bound_runnable1.invoke({"bar": "hello"}))

# Rrunnable fallback
from langchain_core.runnables import RunnableLambda
runnable1 = RunnableLambda(lambda x: x + "foo")
runnable2 = RunnableLambda(lambda x: str(x) + "foo")
chain = runnable1.with_fallbacks([runnable2])
print("Fallback:", chain.invoke(5))

# Runnable retry
from langchain_core.runnables import RunnableLambda
counter = -1
def func4(x):
    global counter
    counter += 1
    print(f"attempt with {counter=}")
    return x / counter
chain = RunnableLambda(func4).with_retry(stop_after_attempt=2)
print("Retry:", chain.invoke(2))


invoke: 5
batch: ['7', '8', '9']
stream:
0
1
2
3
4
compose with pipe: [{'foo': 2}, {'foo': 2}]
RunnableParallel: {'first': {'foo': 2}, 'second': [2, 2]}
RunnableLambda: 7
Merge input and output: {'foo': 10, 'bar': 17}
Bind: {'bar': 'hello', 'foo': 'bye'}
Fallback: 5foo
attempt with counter=0
attempt with counter=1
Retry: 2.0


**RunnableConfig**

In [ ]:
# Runnable config
from langchain_core.runnables import RunnableLambda, RunnableParallel
runnable1 = RunnableLambda(lambda x: {"foo": x})
runnable2 = RunnableLambda(lambda x: [x] * 2)
runnable3 = RunnableLambda(lambda x: str(x))
chain = RunnableParallel(first=runnable1, second=runnable2, third=runnable3)
print("Config:", chain.invoke(7, config={"max_concurrency": 2}))

# Runnable default config
from langchain_core.runnables import RunnableLambda, RunnableParallel
runnable1 = RunnableLambda(lambda x: {"foo": x})
runnable2 = RunnableLambda(lambda x: [x] * 2)
runnable3 = RunnableLambda(lambda x: str(x))
chain = RunnableParallel(first=runnable1, second=runnable2, third=runnable3)
configured_chain = chain.with_config(max_concurrency=2)
print("Default config:", chain.invoke(7))

# Runnable attributes configurable
from typing import Any
from langchain_core.runnables import (
    ConfigurableField, RunnableConfig, RunnableSerializable,
)
class FooRunnable(RunnableSerializable[dict, dict]):
    output_key: str
    def invoke(self,
        input: Any,
        config: RunnableConfig|None = None,
        **kwargs: Any) -> list:
        return self._call_with_config(self.subtract_seven, input, config, **kwargs)
    def subtract_seven(self, input: dict) -> dict:
        return {self.output_key: input["foo"] - 7}

runnable1 = FooRunnable(output_key="bar")
configurable_runnable1 = runnable1.configurable_fields(
    output_key=ConfigurableField(id="output_key")
)
result1 = configurable_runnable1.invoke(
    {"foo": 10}, config={"configurable": {"output_key": "not bar"}}
)
result2 = configurable_runnable1.invoke({"foo": 10})
print("w/ configurable config :", result1)
print("w/o configurable config:", result2)

# Chain components configurable
from typing import Any
from langchain_core.runnables import (
    RunnableConfig, RunnableLambda, RunnableParallel
)
class ListRunnable(RunnableSerializable[Any, list]):
    def invoke(
        self, input: Any, config: RunnableConfig|None = None, **kwargs: Any
    ) -> list:
        return self._call_with_config(self.listify, input, config, **kwargs)
    def listify(self, input: Any) -> list:
        return [input]

class StrRunnable(RunnableSerializable[Any, str]):
    def invoke(
        self, input: Any, config: RunnableConfig|None = None, **kwargs: Any
    ) -> list:
        return self._call_with_config(self.strify, input, config, **kwargs)
    def strify(self, input: Any) -> str:
        return str(input)

"""ListRunnable can be replaced by StrRunnable based on second_step"""
runnable1 = RunnableLambda(lambda x: {"foo": x})
configurable_runnable = ListRunnable().configurable_alternatives(
    ConfigurableField(id="second_step"), default_key="list", string=StrRunnable()
)
chain = runnable1 | configurable_runnable
result1 = chain.invoke(7, config={"configurable": {"second_step": "string"}})
result2 = chain.invoke(7)
print("w/ configurable config :", result1)
print("w/o configurable config:", result2)

# Build chain dynamicall based on input
from langchain_core.runnables import RunnableLambda
runnable1 = RunnableLambda(lambda x: {"foo": x})
runnable2 = RunnableLambda(lambda x: [x] * 2)
chain = RunnableLambda(lambda x: runnable1 if x > 6 else runnable2)
print("Dynamic chain:", chain.invoke(7))

Config: {'first': {'foo': 7}, 'second': [7, 7], 'third': '7'}
Default config: {'first': {'foo': 7}, 'second': [7, 7], 'third': '7'}
w/ configurable config : {'not bar': 3}
w/o configurable config: {'bar': 3}
w/ configurable config : {'foo': 7}
w/o configurable config: [{'foo': 7}]


**Others**

In [21]:
# astream
import nest_asyncio
nest_asyncio.apply()

from langchain_core.runnables import RunnableLambda
runnable1 = RunnableLambda(lambda x: {"foo": x}, name="first")
async def func(x):
    for _ in range(5):
        yield x
runnable2 = RunnableLambda(func, name="second")
chain = runnable1 | runnable2
print("astream:")
async for event in chain.astream_events("bar", version="v2"):
    print(f"event={event['event']} | name={event['name']} | data={event['data']}")

# batch as completed
import time
from langchain_core.runnables import RunnableLambda, RunnableParallel
runnable1 = RunnableLambda(lambda x: time.sleep(x) or print(f"slept {x}"))
print("batch as completed:")
for idx, result in runnable1.batch_as_completed([2, 1]):
    print(idx, result)

# returns subset of output dict
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
runnable1 = RunnableLambda(lambda x: x["baz"] + 5)
chain = RunnablePassthrough.assign(foo=runnable1).pick(["foo", "bar"])
print("subset:", chain.invoke({"bar": "hi", "baz": 2}))

# Make batch from mutiple output
from langchain_core.runnables import RunnableLambda
runnable1 = RunnableLambda(lambda x: list(range(x)))
runnable2 = RunnableLambda(lambda x: x + 5)
chain = runnable1 | runnable2.map()
print("map:", chain.invoke(3))

# Graph
from langchain_core.runnables import RunnableLambda, RunnableParallel
runnable1 = RunnableLambda(lambda x: {"foo": x})
runnable2 = RunnableLambda(lambda x: [x] * 2)
runnable3 = RunnableLambda(lambda x: str(x))
chain = runnable1 | RunnableParallel(second=runnable2, third=runnable3)
chain.get_graph().draw_mermaid()

# Get all prompts
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
prompt1 = ChatPromptTemplate.from_messages(
    [("system", "good ai"), ("human", "{input}")]
)
prompt2 = ChatPromptTemplate.from_messages(
    [
        ("system", "really good ai"),
        ("human", "{input}"),
        ("ai", "{ai_output}"),
        ("human", "{input2}"),
    ]
)
fake_llm = RunnableLambda(lambda prompt: "i am good ai")
chain = prompt1.assign(ai_output=fake_llm) | prompt2 | fake_llm
print("get all prompts:")
for i, prompt in enumerate(chain.get_prompts()):
    print(f"**prompt {i=}**\n")
    print(prompt.pretty_repr())
    print("\n" * 3)

# Listener
import time
from langchain_core.runnables import RunnableLambda
from langchain_core.tracers.schemas import Run
def on_start(run_obj: Run):
    print("start_time:", run_obj.start_time)
def on_end(run_obj: Run):
    print("end_time:", run_obj.end_time)

runnable1 = RunnableLambda(lambda x: time.sleep(x))
print("listner:")
chain = runnable1.with_listeners(on_start=on_start, on_end=on_end)
chain.invoke(2)

astream:
event=on_chain_start | name=RunnableSequence | data={'input': 'bar'}
event=on_chain_start | name=first | data={}
event=on_chain_stream | name=first | data={'chunk': {'foo': 'bar'}}
event=on_chain_start | name=second | data={}
event=on_chain_end | name=first | data={'output': {'foo': 'bar'}, 'input': 'bar'}
event=on_chain_stream | name=second | data={'chunk': {'foo': 'bar'}}
event=on_chain_stream | name=RunnableSequence | data={'chunk': {'foo': 'bar'}}
event=on_chain_stream | name=second | data={'chunk': {'foo': 'bar'}}
event=on_chain_stream | name=RunnableSequence | data={'chunk': {'foo': 'bar'}}
event=on_chain_stream | name=second | data={'chunk': {'foo': 'bar'}}
event=on_chain_stream | name=RunnableSequence | data={'chunk': {'foo': 'bar'}}
event=on_chain_stream | name=second | data={'chunk': {'foo': 'bar'}}
event=on_chain_stream | name=RunnableSequence | data={'chunk': {'foo': 'bar'}}
event=on_chain_stream | name=second | data={'chunk': {'foo': 'bar'}}
event=on_chain_stream 

## Output Parser

> https://python.langchain.com/docs/concepts/output_parsers/

Output parser takes output of a model and transforming it to structured output  for downstream tasks. Useful when you are using LLMs to generate structured data, or to normalize output from chat models and LLMs.

- StrOutputParser
- JsonOutputParser
- XMLOutputParser
- CommaSeparatedListOutputParser
- OutputFixingParser
- RetryWithErrorOutputParser
- PydanticOutputParser
- YamlOutputParser
- PandasDataFrameOutputParser
- EnumOutputParser
- DatetimeOutputParser
- StructuredOutputParser